# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
 pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_file = "cities.csv"

weather_df = pd.read_csv(cities_file)



In [4]:

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bredasdorp,0,ZA,1615070464,77,-34.5322,20.0403,64.40,8.05
1,Ponta do Sol,40,PT,1615070490,67,32.6667,-17.1000,60.80,19.57
2,Avarua,75,CK,1615070468,78,-21.2078,-159.7750,80.60,8.05
3,Jamestown,90,US,1615070470,95,42.0970,-79.2353,26.01,9.89
4,Moberly,1,US,1615070710,33,39.4184,-92.4382,60.80,6.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
59,Beloha,0,MG,1615070581,91,-25.1667,45.0500,70.07,3.44
65,Cockburn Town,0,TC,1615070413,70,21.4612,-71.1419,78.35,5.91
70,Makkah al Mukarramah,0,SA,1615070476,40,21.4267,39.8261,76.98,3.11
84,Hambantota,0,LK,1615070489,100,6.1241,81.1185,73.40,2.30
100,Cidreira,0,BR,1615070496,71,-30.1811,-50.2056,70.93,7.70
102,Biltine,0,TD,1615070567,19,14.5333,20.9167,72.64,9.57
103,Salalah,0,OM,1615070058,69,17.0151,54.0924,75.20,3.44
166,Villa María,0,AR,1615070187,71,-32.4075,-63.2402,79.00,7.72
215,Cabedelo,0,BR,1615070367,89,-6.9811,-34.8339,78.80,5.75
226,Verāval,0,IN,1615070183,80,20.9000,70.3667,73.45,9.46


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 59: Beloha.
Missing field/result... skipping.
------------
Retrieving Results for Index 65: Cockburn Town.
Closest hotel is Osprey Beach Hotel.
------------
Retrieving Results for Index 70: Makkah al Mukarramah.
Closest hotel is Makkah Clock Royal Tower, A Fairmont Hotel.
------------
Retrieving Results for Index 84: Hambantota.
Closest hotel is Bungalow 63.
------------
Retrieving Results for Index 100: Cidreira.
Closest hotel is Hotel Castelo.
------------
Retrieving Results for Index 102: Biltine.
Closest hotel is Hôtel Trois Étoiles de Biltine.
------------
Retrieving Results for Index 103: Salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH.
------------
Retrieving Results for Index 166: Villa María.
Closest hotel is Hotel Milenium.
------------
Retrieving Results for Index 215: Cabedelo.
Closest hotel is Hotel Pigalle.
------------
Retrieving Results for Index 226: Verāval.
Closest hotel is Lords Inn Somnath.
------------
Retrieving Results for Index

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
59,Beloha,0,MG,1615070581,91,-25.1667,45.0500,70.07,3.44,NaN
65,Cockburn Town,0,TC,1615070413,70,21.4612,-71.1419,78.35,5.91,Osprey Beach Hotel
70,Makkah al Mukarramah,0,SA,1615070476,40,21.4267,39.8261,76.98,3.11,"Makkah Clock Royal Tower, A Fairmont Hotel"
84,Hambantota,0,LK,1615070489,100,6.1241,81.1185,73.40,2.30,Bungalow 63
100,Cidreira,0,BR,1615070496,71,-30.1811,-50.2056,70.93,7.70,Hotel Castelo
102,Biltine,0,TD,1615070567,19,14.5333,20.9167,72.64,9.57,Hôtel Trois Étoiles de Biltine
103,Salalah,0,OM,1615070058,69,17.0151,54.0924,75.20,3.44,HAMDAN PLAZA HOTEL SALALAH
166,Villa María,0,AR,1615070187,71,-32.4075,-63.2402,79.00,7.72,Hotel Milenium
215,Cabedelo,0,BR,1615070367,89,-6.9811,-34.8339,78.80,5.75,Hotel Pigalle
226,Verāval,0,IN,1615070183,80,20.9000,70.3667,73.45,9.46,Lords Inn Somnath


In [13]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
